Install necessary dependencies.

In [ ]:
!pip install scikit-learn datasets
!pip install torch --index-url https://download.pytorch.org/whl/cu117 # for cuda

Import all needed packages.

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from torchvision.transforms import Normalize, Compose, Resize, ToTensor
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

Download dataset, remove unnecessary columns and add label 0 to real images and 1 to generated images.
Then split the dataset into a train (80%), validation (10%) and test dataset (10%).

In [ ]:
def get_dataset():
    # datasets
    fake = load_dataset('poloclub/diffusiondb', '2m_random_10k', split='train', data_dir='./')
    real = load_dataset('frgfm/imagenette', '320px', split='train+validation', data_dir='./')

    # remove unnecessary columns
    fake = fake.remove_columns(
        ['prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw',
         'prompt_nsfw'])
    real = real.remove_columns('label')

    # add label column with 0 for real images and for 1 for generated images
    fake = fake.map(lambda x: {'image': x['image'], 'label': 1})
    real = real.map(lambda x: {'image': x['image'], 'label': 0})

    # split fake dataset into train, validation and test sets
    fake_train_testvalid = fake.train_test_split(test_size=0.2)
    fake_test_valid = fake_train_testvalid['test'].train_test_split(test_size=0.5)

    # split real dataset into train, validation and test sets
    real_train_testvalid = real.train_test_split(test_size=0.2)
    real_test_valid = real_train_testvalid['test'].train_test_split(test_size=0.5)

    # combine fake and real datasets into single dataset for each split
    train_dataset = concatenate_datasets([fake_train_testvalid['train'], real_train_testvalid['train']])
    val_dataset = concatenate_datasets([fake_test_valid['train'], real_test_valid['train']])
    test_dataset = concatenate_datasets([fake_test_valid['test'], real_test_valid['test']])

    return train_dataset, val_dataset, test_dataset

Transform and preprocess test data. Convert the images to pixel_values (tensors).

In [ ]:
test_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

def transform_test(batch):
    batch['pixel_values'] = [test_transforms(img.convert("RGB")) for img in batch['image']]
    del batch['image']
    return batch

Functions for prediction.

In [ ]:
def get_image_label(data, device):
    """
    Get image and label data. Convert label data to the same nn output shape.
    """
    image = data['pixel_values'].to(device)
    label = data['label'].unsqueeze(1).float().to(device)

    return image, label


def predict_test(test_loader, model, device):
    """
    Predict all the test dataset and calculate confusion matrix, AUROC and classification report with f1 score, precision and recall.
    """
    predictions, targets = [], []

    model.eval()
    for data in test_loader:
        image, label = get_image_label(data, device)

        output = torch.sigmoid(model(image))
        preds = torch.round(output)

        preds = preds.detach().cpu().numpy()
        label = label.detach().cpu().numpy()

        for i in range(len(preds)):
            predictions.append(preds[i])
            targets.append((label[i]))

    predictions = np.array(predictions)
    targets = np.array(targets)

    conf_mat = confusion_matrix(targets, predictions)
    class_rep = classification_report(targets, predictions, target_names=('Real', 'Generated'))
    auroc = roc_auc_score(targets, predictions)

    print(conf_mat)
    print(class_rep)
    print(auroc)

    file = open(f'./model_data/resnet50_metrics.txt', 'w')
    file.write(f'ResNet50 Dataset1\n\nConfusion Matrix\n\n{conf_mat}\n\nClassification Report\n\n{class_rep}\n\n'
               f'AUROC: {auroc}')
    file.close()

Execute Prediction.

In [ ]:
# used device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, 'will be used.')

# get the dataset
_, _, test_dataset = get_dataset()

# apply transforms and preprocessing to dataset
test_dataset.set_transform(transform_test)

# load data
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# load best model
model = torch.jit.load('./model_data/resnet50.pth')
model.eval()

predict_test(test_loader, model, device)